In [1]:
%config IPCompleter.greedy=True

In [2]:
%config IPCompleter.use_jedi = True

<h3><center>Kinopoisk TOP-250 Movies Page Parsing<center><br>
    using Requests, BeautifulSoup4, Selenium

Natalia Suchkova - 25.01.2023

In [3]:
import requests
from bs4 import BeautifulSoup
import selenium
from selenium.common.exceptions import NoSuchElementException


from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [4]:
from lxml import etree
from fake_useragent import UserAgent

In [5]:
import pandas as pd
import re
import unicodedata

import time

### Тестовый парсинг на одном эксземпляре

In [6]:
url = 'https://www.kinopoisk.ru/lists/movies/top250/?page=1'

In [8]:
r = requests.get(url)
r.text

'<!DOCTYPE html><html><head><meta name="viewport" content="width=device-width"/><meta charSet="utf-8"/><title data-tid="57f72b5">250 лучших фильмов — Кинопоиск</title><meta name="theme-color" content="#1f1f1f" data-tid="57f72b5"/><meta name="apple-mobile-web-app-capable" content="yes" data-tid="57f72b5"/><meta name="apple-mobile-web-app-status-bar-style" content="black" data-tid="57f72b5"/><meta name="apple-mobile-web-app-title" content="Кинопоиск" data-tid="57f72b5"/><meta name="apple-itunes-app" content="app-id=477718890, ct=kp-web, pt=214944, mt=8" data-tid="57f72b5"/><meta name="application-name" content="Кинопоиск" data-tid="57f72b5"/><meta property="fb:app_id" content="121953784483000" data-tid="57f72b5"/><meta property="fb:pages" content="152308956519" data-tid="57f72b5"/><meta property="og:site_name" content="Кинопоиск" data-tid="57f72b5"/><meta name="msapplication-TileColor" content="#000" data-tid="57f72b5"/><meta name="msapplication-TileImage" content="https://yastatic.net/s

In [9]:
soup = BeautifulSoup(r.text, features='lxml')
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <meta content="width=device-width" name="viewport"/>
  <meta charset="utf-8"/>
  <title data-tid="57f72b5">
   250 лучших фильмов — Кинопоиск
  </title>
  <meta content="#1f1f1f" data-tid="57f72b5" name="theme-color"/>
  <meta content="yes" data-tid="57f72b5" name="apple-mobile-web-app-capable"/>
  <meta content="black" data-tid="57f72b5" name="apple-mobile-web-app-status-bar-style"/>
  <meta content="Кинопоиск" data-tid="57f72b5" name="apple-mobile-web-app-title"/>
  <meta content="app-id=477718890, ct=kp-web, pt=214944, mt=8" data-tid="57f72b5" name="apple-itunes-app"/>
  <meta content="Кинопоиск" data-tid="57f72b5" name="application-name"/>
  <meta content="121953784483000" data-tid="57f72b5" property="fb:app_id"/>
  <meta content="152308956519" data-tid="57f72b5" property="fb:pages"/>
  <meta content="Кинопоиск" data-tid="57f72b5" property="og:site_name"/>
  <meta content="#000" data-tid="57f72b5" name="msapplication-TileColor"/>
  <meta content="ht

In [10]:
link = 'https://www.kinopoisk.ru' + soup.find('div', class_='styles_upper__j8BIs').find('a', class_='base-movie-main-info_link__YwtP1').get('href')

In [11]:
link

'https://www.kinopoisk.ru/film/435/'

In [12]:
name = soup.find('div', class_='styles_upper__j8BIs').find('span', class_='styles_mainTitle__IFQyZ styles_activeMovieTittle__kJdJj').text
name

'Зеленая миля'

In [13]:
name_eng = soup.find('div', class_='styles_upper__j8BIs').find('span', class_='desktop-list-main-info_secondaryTitle__ighTt').text
name_eng

'The Green Mile'

In [14]:
for i in soup.find('div', class_='styles_upper__j8BIs').find_all('span', class_='desktop-list-main-info_truncatedText__IMQRP'):
    print(i.get_text(strip=True))
   # print(unicodedata.normalize("NFKD", i.text))

США • драма  Режиссёр: Фрэнк Дарабонт
В ролях: Том Хэнкс, Дэвид Морс


In [15]:
soup.find('div', class_='styles_upper__j8BIs').find_all('span', class_='desktop-list-main-info_truncatedText__IMQRP')

[<span class="desktop-list-main-info_truncatedText__IMQRP">США • драма  Режиссёр: Фрэнк Дарабонт</span>,
 <span class="desktop-list-main-info_truncatedText__IMQRP">В ролях: Том Хэнкс, Дэвид Морс</span>]

In [16]:
string = unicodedata.normalize("NFKD", soup.find('div', class_='styles_upper__j8BIs').find_all('span', class_='desktop-list-main-info_truncatedText__IMQRP')[0].text)
regexp = re.compile("Режиссёр: (.*)$")
director = regexp.search(string).group(1)
director 

'Фрэнк Дарабонт'

In [17]:
string = unicodedata.normalize("NFKD", soup.find('div', class_='styles_upper__j8BIs').find_all('span', class_='desktop-list-main-info_truncatedText__IMQRP')[0].text)
regexp = re.compile('(\w+) •')
country = regexp.search(string).group(1)
country 

'США'

In [18]:
string = unicodedata.normalize("NFKD", soup.find('div', class_='styles_upper__j8BIs').find_all('span', class_='desktop-list-main-info_truncatedText__IMQRP')[0].text)
regexp = re.compile('• (.*)$')
genre = regexp.search(string).group(1)
regexp = re.compile('[^  ]*')
genre = regexp.search(genre).group(0)
genre

'драма'

In [19]:
string = unicodedata.normalize("NFKD", soup.find('div', class_='styles_upper__j8BIs').find('span', class_='desktop-list-main-info_secondaryText__M_aus').text)
print(string)
year = string.split(', ')[1]
minutes_time = re.compile('[0-9]+').search(string.split(', ')[2]).group(0)
print(year, minutes_time)

, 1999, 189 мин.
1999 189


In [20]:
rating = soup.find('div', class_='styles_upper__j8BIs').find('span', class_='styles_kinopoiskValuePositive__vOb2E styles_kinopoiskValue__9qXjg styles_top250Type__mPloU').text
rating

'9.2'

### Объединенный парсинг всех фильмов на 1 странице (из 5ти)

Отметим, что есть отличия при парсинге российских и иностранных фильмов, так как у первых отсутствует название на английском языке. Английское название, год выпуска и продолжительность находятся в одном классе и подклассе

`div class="desktop-list-main-info_secondaryTitleSlot__mc0mI">`

   `<span class="desktop-list-main-info_secondaryTitle__ighTt">The Green Mile</span>`
    
   `<span class="desktop-list-main-info_secondaryText__M_aus">, 1999, 189&nbsp;мин.</span>`
    
`</div`

In [21]:
films = soup.find_all('div', class_='styles_upper__j8BIs')
len(films)

50

In [51]:
link = []
name = []
name_eng = []
rating = []
director = []
country = []
genre = []
year = []
minutes_time = []

num_pages = 5

In [52]:
for page in range(1, num_pages + 1):
    
    url = f'https://www.kinopoisk.ru/lists/movies/top250/?page={page}'
    print(url)
    r = requests.get(url)
    time.sleep(40) 
    soup = BeautifulSoup(r.content, features='lxml')
    time.sleep(40)  # for captcha problem stabilization
    films = soup.find_all('div', class_='styles_upper__j8BIs')
    time.sleep(60)
    print(len(films))  # monitoring url response

    for film in films:
        link.append('https://www.kinopoisk.ru' + film.find('a', class_='base-movie-main-info_link__YwtP1').get('href'))
        name.append(film.find('span', class_='styles_mainTitle__IFQyZ styles_activeMovieTittle__kJdJj').text)

        rating.append(float(
                          film.find('span', 
                          class_='styles_kinopoiskValuePositive__vOb2E styles_kinopoiskValue__9qXjg styles_top250Type__mPloU').text)
                         )

        string = unicodedata.normalize("NFKD", film.find_all('span', 
                                                             class_='desktop-list-main-info_truncatedText__IMQRP')[0].text)
        director.append(re.compile("Режиссёр: (.*)$").search(string).group(1))
        
        time.sleep(1)
        
        string = unicodedata.normalize("NFKD", film.find_all('span', 
                                                             class_='desktop-list-main-info_truncatedText__IMQRP')[0].text)
        country.append(re.compile('(\w+) •').search(string).group(1))

        string = unicodedata.normalize("NFKD", film.find_all('span', 
                                                             class_='desktop-list-main-info_truncatedText__IMQRP')[0].text)
        genre.append(re.compile('[^  ]*').search(re.compile('• (.*)$').search(string).group(1)).group(0))

        time.sleep(2)
        
        data_block = film.find('div', class_='desktop-list-main-info_secondaryTitleSlot__mc0mI').find_all('span')
        
        if len(data_block) == 2:

            name_eng.append(data_block[0].text)
            string = unicodedata.normalize("NFKD", data_block[1].text)
            year.append(int(string.split(', ')[1]))
            minutes_time.append(int(re.compile('[0-9]+').search(string.split(', ')[2]).group(0)))

        else:
            
            name_eng.append('---')
            string = unicodedata.normalize("NFKD", data_block[0].text)
            year.append(int(string.split(', ')[0]))
            minutes_time.append(int(re.compile('[0-9]+').search(string.split(', ')[1]).group(0)))

            
    data = {'Kinopoisk_Link': link, 'Name': name, 'Name_ENG': name_eng,
            'Year': year, 'Country': country, 'Genre': genre,
            'Director': director, 'Length': minutes_time, 'Rating': rating}
    
    time.sleep(60)

https://www.kinopoisk.ru/lists/movies/top250/?page=1
50
https://www.kinopoisk.ru/lists/movies/top250/?page=2
0
https://www.kinopoisk.ru/lists/movies/top250/?page=3
50
https://www.kinopoisk.ru/lists/movies/top250/?page=4
0
https://www.kinopoisk.ru/lists/movies/top250/?page=5
0


In [28]:
data = pd.DataFrame(data)

In [29]:
data

,Kinopoisk_Link,Name,Name_ENG,Year,Country,Genre,Director,Length,Rating
0,https://www.kinopoisk.ru/film/435/,Зеленая миля,The Green Mile,1999,США,драма,Фрэнк Дарабонт,189,9.2
1,https://www.kinopoisk.ru/film/329/,Список Шиндлера,Schindler's List,1993,США,драма,Стивен Спилберг,195,9.1
2,https://www.kinopoisk.ru/film/326/,Побег из Шоушенка,The Shawshank Redemption,1994,США,драма,Фрэнк Дарабонт,142,9.0
3,https://www.kinopoisk.ru/film/448/,Форрест Гамп,Forrest Gump,1994,США,драма,Роберт Земекис,142,9.0
4,https://www.kinopoisk.ru/film/679486/,Тайна Коко,Coco,2017,США,мультфильм,Ли Анкрич,105,8.9
...,...,...,...,...,...,...,...,...,...
245,https://www.kinopoisk.ru/film/278522/,Хоббит: Нежданное путешествие,The Hobbit: An Unexpected Journey,2012,Зеландия,фэнтези,Питер Джексон,169,8.4
246,https://www.kinopoisk.ru/film/441/,Принцесса Мононоке,Mononoke-hime,1997,Япония,аниме,Хаяо Миядзаки,134,8.4
247,https://www.kinopoisk.ru/film/81314/,Отступники,The Departed,2006,США,триллер,Мартин Скорсезе,151,8.4
248,https://www.kinopoisk.ru/film/399/,Храброе сердце,Braveheart,1995,США,история,Мэл Гибсон,178,8.4


### Trying to Get Rid of Captchas and Enhance Navigation

In [34]:
browser = webdriver.Chrome(r'C:\Users\User\Desktop\chromedriver.exe')
url = 'https://www.kinopoisk.ru/lists/movies/top250/'

In [35]:
browser.get(url)

In [36]:
# пробуем листать страницы сайта с селениумом
while True:
    try:
    #     next_page_button = browser.find_element_by_css_selector('#__next > div.styles_root__goB3B > div.styles_middleContainer__vJjLN.styles_baseContainer__bGRTT > div.styles_mainContainer__59YLl > div.styles_contentSlot__h_lSN > main > div.styles_root__AT6_5.styles_root__RoFSb > a.styles_end__aEsmB.styles_start__UvE6T').click()
        next_page_button = browser.find_element_by_xpath('//a[@title="Вперед"]')
        next_page_button.click()
        print(browser.current_url)
    
        # !!!! селениум не может найти по всему классу, видит название класса только до пробела,
        # !!!! потом для него следующий уже второй класс и тд. а он как и суп при поиске останавливается на первом
    except NoSuchElementException:
        print('No Such Element on the Page')
        break

url = browser.current_url
print('showcaptcha' in url)
browser.quit()

https://www.kinopoisk.ru/lists/movies/top250/?page=2
https://www.kinopoisk.ru/lists/movies/top250/?page=3
https://www.kinopoisk.ru/lists/movies/top250/?page=4
https://www.kinopoisk.ru/lists/movies/top250/?page=5
No Such Element on the Page
False


In [161]:
def wait_until(somepredicate, timeout, period=0.5):
    mustend = time.time() + timeout
    while time.time() < mustend:
        if somepredicate(): 
            return True
        else:
            time.sleep(period)
    return False

#### Trace and Autofill Captcha

In [37]:
def check_captcha(browser, url, page_soup):
    
    if 'captcha' in url or page_soup.find('span', class_='Text_typography_body-long-m'): 
        print("Captcha's here!!!!")
        box_1 = browser.find_element_by_xpath('//input[@type="submit"]')
        box_1.click()
        time.sleep(60)

        while 'captcha' in browser.current_url:
            pass
        
    return browser.current_url

In [38]:
link = []
name = []
name_eng = []
rating = []
director = []
country = []
genre = []
year = []
minutes_time = []

In [39]:
%%time
browser = webdriver.Chrome(r'C:\Users\User\Desktop\chromedriver.exe')
url = 'https://www.kinopoisk.ru/lists/movies/top250/'
browser.get(url)
time.sleep(5)

while True:
    
    url = browser.current_url
    time.sleep(3)
   
    url = check_captcha(browser, url, BeautifulSoup(browser.page_source, features='lxml'))
    print(url)
    
    r = requests.get(url)
    time.sleep(2)  # for captcha problem stabilization
    check_captcha(browser, url, BeautifulSoup(browser.page_source, features='lxml'))
    
    soup = BeautifulSoup(r.content, features='lxml')
    print(soup)
    time.sleep(3)  
    films = soup.find_all('div', class_='styles_upper__j8BIs')
    print(len(films))  # monitoring url response

    for film in films:
        link.append('https://www.kinopoisk.ru' + film.find('a', class_='base-movie-main-info_link__YwtP1').get('href'))
        name.append(film.find('span', class_='styles_mainTitle__IFQyZ styles_activeMovieTittle__kJdJj').text)

        rating.append(float(
                          film.find('span', 
                          class_='styles_kinopoiskValuePositive__vOb2E styles_kinopoiskValue__9qXjg styles_top250Type__mPloU').text)
                         )

        string = unicodedata.normalize("NFKD", film.find_all('span', 
                                                             class_='desktop-list-main-info_truncatedText__IMQRP')[0].text)
        director.append(re.compile("Режиссёр: (.*)$").search(string).group(1))
        
        time.sleep(2)
        
        string = unicodedata.normalize("NFKD", film.find_all('span', 
                                                             class_='desktop-list-main-info_truncatedText__IMQRP')[0].text)
        country.append(re.compile('(\w+) •').search(string).group(1))

        string = unicodedata.normalize("NFKD", film.find_all('span', 
                                                             class_='desktop-list-main-info_truncatedText__IMQRP')[0].text)
        genre.append(re.compile('[^  ]*').search(re.compile('• (.*)$').search(string).group(1)).group(0))

        time.sleep(2)
        
        data_block = film.find('div', class_='desktop-list-main-info_secondaryTitleSlot__mc0mI').find_all('span')
        
        if len(data_block) == 2:

            name_eng.append(data_block[0].text)
            string = unicodedata.normalize("NFKD", data_block[1].text)
            year.append(int(string.split(', ')[1]))
            minutes_time.append(int(re.compile('[0-9]+').search(string.split(', ')[2]).group(0)))

        else:
            
            name_eng.append('---')
            string = unicodedata.normalize("NFKD", data_block[0].text)
            year.append(int(string.split(', ')[0]))
            minutes_time.append(int(re.compile('[0-9]+').search(string.split(', ')[1]).group(0)))
    
    data = {'Kinopoisk_Link': link, 'Name': name, 'Name_ENG': name_eng,
            'Year': year, 'Country': country, 'Genre': genre,
            'Director': director, 'Length': minutes_time, 'Rating': rating}

    
    try:  # here we navigate the pages
        next_page_button = browser.find_element_by_xpath('//a[@title="Вперед"]')
        next_page_button.click()
        time.sleep(3)

    except NoSuchElementException:
#         browser.quit()
        break


https://www.kinopoisk.ru/lists/movies/top250/
<!DOCTYPE html>
<html><head><meta content="width=device-width" name="viewport"/><meta charset="utf-8"/><title data-tid="57f72b5">250 лучших фильмов — Кинопоиск</title><meta content="#1f1f1f" data-tid="57f72b5" name="theme-color"/><meta content="yes" data-tid="57f72b5" name="apple-mobile-web-app-capable"/><meta content="black" data-tid="57f72b5" name="apple-mobile-web-app-status-bar-style"/><meta content="Кинопоиск" data-tid="57f72b5" name="apple-mobile-web-app-title"/><meta content="app-id=477718890, ct=kp-web, pt=214944, mt=8" data-tid="57f72b5" name="apple-itunes-app"/><meta content="Кинопоиск" data-tid="57f72b5" name="application-name"/><meta content="121953784483000" data-tid="57f72b5" property="fb:app_id"/><meta content="152308956519" data-tid="57f72b5" property="fb:pages"/><meta content="Кинопоиск" data-tid="57f72b5" property="og:site_name"/><meta content="#000" data-tid="57f72b5" name="msapplication-TileColor"/><meta content="https:/

50
https://www.kinopoisk.ru/lists/movies/top250/?page=2
<!DOCTYPE html>
<html prefix="og: http://ogp.me/ns#"><head><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta charset="utf-8"/><meta content="width=device-width,initial-scale=1" name="viewport"/><title data-react-helmet="true">Ой!</title><meta content="Яндекс" data-react-helmet="true" property="og:title"/><meta content="Найдётся всё" data-react-helmet="true" property="og:description"/><meta content="https://yastatic.net/s3/home-static/_/37/37a02b5dc7a51abac55d8a5b6c865f0e.png" data-react-helmet="true" property="og:image"/><link href="/captcha_smart.0bfc4d9561e7e6e94795.min.css?k=1666276736660" rel="stylesheet"/><style>@media only screen and (min-width:651px){body{background-image:url('https://captcha-backgrounds.s3.yandex.net/static/kinopoisk-background.jpg')}}.LogoLink{background-image:url('')}</style></head><body><div id="root"><div class="Theme Theme_color_yandex-default Theme_root_default"><div class="Container"><div 

0
https://www.kinopoisk.ru/lists/movies/top250/?page=3
<!DOCTYPE html>
<html><head><meta content="width=device-width" name="viewport"/><meta charset="utf-8"/><title data-tid="57f72b5">250 лучших фильмов — Кинопоиск</title><meta content="#1f1f1f" data-tid="57f72b5" name="theme-color"/><meta content="yes" data-tid="57f72b5" name="apple-mobile-web-app-capable"/><meta content="black" data-tid="57f72b5" name="apple-mobile-web-app-status-bar-style"/><meta content="Кинопоиск" data-tid="57f72b5" name="apple-mobile-web-app-title"/><meta content="app-id=477718890, ct=kp-web, pt=214944, mt=8" data-tid="57f72b5" name="apple-itunes-app"/><meta content="Кинопоиск" data-tid="57f72b5" name="application-name"/><meta content="121953784483000" data-tid="57f72b5" property="fb:app_id"/><meta content="152308956519" data-tid="57f72b5" property="fb:pages"/><meta content="Кинопоиск" data-tid="57f72b5" property="og:site_name"/><meta content="#000" data-tid="57f72b5" name="msapplication-TileColor"/><meta content

50
https://www.kinopoisk.ru/lists/movies/top250/?page=4
<!DOCTYPE html>
<html prefix="og: http://ogp.me/ns#"><head><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta charset="utf-8"/><meta content="width=device-width,initial-scale=1" name="viewport"/><title data-react-helmet="true">Ой!</title><meta content="Яндекс" data-react-helmet="true" property="og:title"/><meta content="Найдётся всё" data-react-helmet="true" property="og:description"/><meta content="https://yastatic.net/s3/home-static/_/37/37a02b5dc7a51abac55d8a5b6c865f0e.png" data-react-helmet="true" property="og:image"/><link href="/captcha_smart.0bfc4d9561e7e6e94795.min.css?k=1666276736660" rel="stylesheet"/><style>@media only screen and (min-width:651px){body{background-image:url('https://captcha-backgrounds.s3.yandex.net/static/kinopoisk-background.jpg')}}.LogoLink{background-image:url('')}</style></head><body><div id="root"><div class="Theme Theme_color_yandex-default Theme_root_default"><div class="Container"><div 

0
https://www.kinopoisk.ru/lists/movies/top250/?page=5
<!DOCTYPE html>
<html prefix="og: http://ogp.me/ns#"><head><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta charset="utf-8"/><meta content="width=device-width,initial-scale=1" name="viewport"/><title data-react-helmet="true">Ой!</title><meta content="Яндекс" data-react-helmet="true" property="og:title"/><meta content="Найдётся всё" data-react-helmet="true" property="og:description"/><meta content="https://yastatic.net/s3/home-static/_/37/37a02b5dc7a51abac55d8a5b6c865f0e.png" data-react-helmet="true" property="og:image"/><link href="/captcha_smart.0bfc4d9561e7e6e94795.min.css?k=1666276736660" rel="stylesheet"/><style>@media only screen and (min-width:651px){body{background-image:url('https://captcha-backgrounds.s3.yandex.net/static/kinopoisk-background.jpg')}}.LogoLink{background-image:url('')}</style></head><body><div id="root"><div class="Theme Theme_color_yandex-default Theme_root_default"><div class="Container"><div c

0
Wall time: 7min 50s


In [204]:
data = pd.DataFrame(data)

In [205]:
data

,Kinopoisk_Link,Name,Name_ENG,Year,Country,Genre,Director,Length,Rating
0,https://www.kinopoisk.ru/film/435/,Зеленая миля,The Green Mile,1999,США,драма,Фрэнк Дарабонт,189,9.2
1,https://www.kinopoisk.ru/film/329/,Список Шиндлера,Schindler's List,1993,США,драма,Стивен Спилберг,195,9.1
2,https://www.kinopoisk.ru/film/326/,Побег из Шоушенка,The Shawshank Redemption,1994,США,драма,Фрэнк Дарабонт,142,9.0
3,https://www.kinopoisk.ru/film/448/,Форрест Гамп,Forrest Gump,1994,США,драма,Роберт Земекис,142,9.0
4,https://www.kinopoisk.ru/film/679486/,Тайна Коко,Coco,2017,США,мультфильм,Ли Анкрич,105,8.9
...,...,...,...,...,...,...,...,...,...
195,https://www.kinopoisk.ru/film/461939/,Линкольн для адвоката,The Lincoln Lawyer,2011,США,криминал,Брэд Фурман,114,8.2
196,https://www.kinopoisk.ru/film/81041/,Алеша Попович и Тугарин Змей,---,2004,Россия,мультфильм,Константин Бронзит,79,8.2
197,https://www.kinopoisk.ru/film/81555/,Загадочная история Бенджамина Баттона,The Curious Case of Benjamin Button,2008,США,драма,Дэвид Финчер,166,8.2
198,https://www.kinopoisk.ru/film/333/,Звёздные войны: Эпизод 4 — Новая надежда,Star Wars,1977,США,фантастика,Джордж Лукас,121,8.1


#### Fake UserAgent

In [43]:
options = Options()
options.add_argument("window-size=1400,1000")

ua = UserAgent()

Если у вас возникла такая-же проблема как у меня, а именно капча при парсинге сайта, то прочитайте как я её решил.

Изначально я использовал библиотеки requests (для отправки запросов к сайту) и bs4 (для самого парсинга).
Сначала я сделал задержку, если программа встречает капчу, то она останавливает свою работу на 1 час, и дальше продолжает работать. Это не сработало, ни через час, ни через два, ни через 3 дня.
Дальше я думал подключить selenium, и когда выходит капча, открывать эту страничку с помощью selenium, там проходить капчу вручную, закрывать selenium, и пробовать через requests еще раз отправлять запросы на страницу. Капча все равно осталась.
Переписать парсер под selenium - без использования requests и bs4, хорошее решение, но не всегда подходит. В моем случае было слишком муторно использовать это, поэтому я обратился сюда.
Тут мне подсказали такой вариант: использовать сессии (requests.Session()) и очищать куки при смене IP (кстати да, я подключал библиотеки для смены IP через proxy, а так же библиотека для генерации fake-User-Agent) и передавать в заголовки параметр referer. Возможно это и работает, но не для меня. Капча все равно осталась.

Ну и вот собственно само решение :)
Я решил заменить библиотеку requests на selenium. Я через него подключался к сайту (у меня даже открывалось окно браузера) и с помощью метода page_source я получал html-код странички. Кстати, будьте внимательными, я так понял что с помощью этого метода можно получить только html, без js и css. Поэтому если сайт использует js для генерации контента, то у вас вряд-ли это получится. А потом, я просто с помощью bs4 парсил полученную html страничку, и извлекал из нее нужные мне данные. Да, капча все равно появлялась, но она появилась лишь однажды, я её вручную решил прямо через окно selenium, и можно сказать что я обошел капчу, так как она больше в процессе парсинга у меня не выскакивала.

In [53]:
link = []
name = []
name_eng = []
rating = []
director = []
country = []
genre = []
year = []
minutes_time = []

In [54]:
browser = webdriver.Chrome(options=options, executable_path=r'C:\Users\User\Desktop\chromedriver.exe')
url = 'https://www.kinopoisk.ru/lists/movies/top250/'
browser.get(url)
time.sleep(10)

while True:
    
    url = browser.current_url
    if url[:-1] == '4':
        user_agent = ua.random
        options.add_argument(f'user-agent={user_agent}')
        print(user_agent)
    
    time.sleep(6)
   
    url = check_captcha(browser, url, BeautifulSoup(browser.page_source, features='lxml'))
    print(url)
    
    r = requests.get(url)
    check_captcha(browser, url, BeautifulSoup(browser.page_source, features='lxml'))
    time.sleep(20)  # for captcha problem stabilization
    
    soup = BeautifulSoup(r.content, features='lxml')
    print(soup)
    time.sleep(40)  
    films = soup.find_all('div', class_='styles_upper__j8BIs')
    print(len(films))  # monitoring url response

    for film in films:
        link.append('https://www.kinopoisk.ru' + film.find('a', class_='base-movie-main-info_link__YwtP1').get('href'))
        name.append(film.find('span', class_='styles_mainTitle__IFQyZ styles_activeMovieTittle__kJdJj').text)

        rating.append(float(
                          film.find('span', 
                          class_='styles_kinopoiskValuePositive__vOb2E styles_kinopoiskValue__9qXjg styles_top250Type__mPloU').text)
                         )

        string = unicodedata.normalize("NFKD", film.find_all('span', 
                                                             class_='desktop-list-main-info_truncatedText__IMQRP')[0].text)
        director.append(re.compile("Режиссёр: (.*)$").search(string).group(1))
        
        time.sleep(2)
        
        string = unicodedata.normalize("NFKD", film.find_all('span', 
                                                             class_='desktop-list-main-info_truncatedText__IMQRP')[0].text)
        country.append(re.compile('(\w+) •').search(string).group(1))

        string = unicodedata.normalize("NFKD", film.find_all('span', 
                                                             class_='desktop-list-main-info_truncatedText__IMQRP')[0].text)
        genre.append(re.compile('[^  ]*').search(re.compile('• (.*)$').search(string).group(1)).group(0))

        time.sleep(4)
        
        data_block = film.find('div', class_='desktop-list-main-info_secondaryTitleSlot__mc0mI').find_all('span')
        
        if len(data_block) == 2:

            name_eng.append(data_block[0].text)
            string = unicodedata.normalize("NFKD", data_block[1].text)
            year.append(int(string.split(', ')[1]))
            minutes_time.append(int(re.compile('[0-9]+').search(string.split(', ')[2]).group(0)))

        else:
            
            name_eng.append('---')
            string = unicodedata.normalize("NFKD", data_block[0].text)
            year.append(int(string.split(', ')[0]))
            minutes_time.append(int(re.compile('[0-9]+').search(string.split(', ')[1]).group(0)))
    
    data = {'Kinopoisk_Link': link, 'Name': name, 'Name_ENG': name_eng,
            'Year': year, 'Country': country, 'Genre': genre,
            'Director': director, 'Length': minutes_time, 'Rating': rating}

    time.sleep(60)
    try:  # here we navigate the pages
        next_page_button = browser.find_element_by_xpath('//a[@title="Вперед"]')
        next_page_button.click()
        time.sleep(5)

    except NoSuchElementException:
        browser.quit()
        break

https://www.kinopoisk.ru/lists/movies/top250/
<!DOCTYPE html>
<html><head><meta content="width=device-width" name="viewport"/><meta charset="utf-8"/><title data-tid="57f72b5">250 лучших фильмов — Кинопоиск</title><meta content="#1f1f1f" data-tid="57f72b5" name="theme-color"/><meta content="yes" data-tid="57f72b5" name="apple-mobile-web-app-capable"/><meta content="black" data-tid="57f72b5" name="apple-mobile-web-app-status-bar-style"/><meta content="Кинопоиск" data-tid="57f72b5" name="apple-mobile-web-app-title"/><meta content="app-id=477718890, ct=kp-web, pt=214944, mt=8" data-tid="57f72b5" name="apple-itunes-app"/><meta content="Кинопоиск" data-tid="57f72b5" name="application-name"/><meta content="121953784483000" data-tid="57f72b5" property="fb:app_id"/><meta content="152308956519" data-tid="57f72b5" property="fb:pages"/><meta content="Кинопоиск" data-tid="57f72b5" property="og:site_name"/><meta content="#000" data-tid="57f72b5" name="msapplication-TileColor"/><meta content="https:/

50
https://www.kinopoisk.ru/lists/movies/top250/?page=2
<!DOCTYPE html>
<html><head><meta content="width=device-width" name="viewport"/><meta charset="utf-8"/><title data-tid="57f72b5">250 лучших фильмов — Кинопоиск</title><meta content="#1f1f1f" data-tid="57f72b5" name="theme-color"/><meta content="yes" data-tid="57f72b5" name="apple-mobile-web-app-capable"/><meta content="black" data-tid="57f72b5" name="apple-mobile-web-app-status-bar-style"/><meta content="Кинопоиск" data-tid="57f72b5" name="apple-mobile-web-app-title"/><meta content="app-id=477718890, ct=kp-web, pt=214944, mt=8" data-tid="57f72b5" name="apple-itunes-app"/><meta content="Кинопоиск" data-tid="57f72b5" name="application-name"/><meta content="121953784483000" data-tid="57f72b5" property="fb:app_id"/><meta content="152308956519" data-tid="57f72b5" property="fb:pages"/><meta content="Кинопоиск" data-tid="57f72b5" property="og:site_name"/><meta content="#000" data-tid="57f72b5" name="msapplication-TileColor"/><meta conten

50
https://www.kinopoisk.ru/lists/movies/top250/?page=3
<!DOCTYPE html>
<html prefix="og: http://ogp.me/ns#"><head><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta charset="utf-8"/><meta content="width=device-width,initial-scale=1" name="viewport"/><title data-react-helmet="true">Ой!</title><meta content="Яндекс" data-react-helmet="true" property="og:title"/><meta content="Найдётся всё" data-react-helmet="true" property="og:description"/><meta content="https://yastatic.net/s3/home-static/_/37/37a02b5dc7a51abac55d8a5b6c865f0e.png" data-react-helmet="true" property="og:image"/><link href="/captcha_smart.0bfc4d9561e7e6e94795.min.css?k=1666276736660" rel="stylesheet"/><style>@media only screen and (min-width:651px){body{background-image:url('https://captcha-backgrounds.s3.yandex.net/static/kinopoisk-background.jpg')}}.LogoLink{background-image:url('')}</style></head><body><div id="root"><div class="Theme Theme_color_yandex-default Theme_root_default"><div class="Container"><div 

0
https://www.kinopoisk.ru/lists/movies/top250/?page=4
<!DOCTYPE html>
<html prefix="og: http://ogp.me/ns#"><head><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta charset="utf-8"/><meta content="width=device-width,initial-scale=1" name="viewport"/><title data-react-helmet="true">Ой!</title><meta content="Яндекс" data-react-helmet="true" property="og:title"/><meta content="Найдётся всё" data-react-helmet="true" property="og:description"/><meta content="https://yastatic.net/s3/home-static/_/37/37a02b5dc7a51abac55d8a5b6c865f0e.png" data-react-helmet="true" property="og:image"/><link href="/captcha_smart.0bfc4d9561e7e6e94795.min.css?k=1666276736660" rel="stylesheet"/><style>@media only screen and (min-width:651px){body{background-image:url('https://captcha-backgrounds.s3.yandex.net/static/kinopoisk-background.jpg')}}.LogoLink{background-image:url('')}</style></head><body><div id="root"><div class="Theme Theme_color_yandex-default Theme_root_default"><div class="Container"><div c

0
https://www.kinopoisk.ru/lists/movies/top250/?page=5
<!DOCTYPE html>
<html prefix="og: http://ogp.me/ns#"><head><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta charset="utf-8"/><meta content="width=device-width,initial-scale=1" name="viewport"/><title data-react-helmet="true">Ой!</title><meta content="Яндекс" data-react-helmet="true" property="og:title"/><meta content="Найдётся всё" data-react-helmet="true" property="og:description"/><meta content="https://yastatic.net/s3/home-static/_/37/37a02b5dc7a51abac55d8a5b6c865f0e.png" data-react-helmet="true" property="og:image"/><link href="/captcha_smart.0bfc4d9561e7e6e94795.min.css?k=1666276736660" rel="stylesheet"/><style>@media only screen and (min-width:651px){body{background-image:url('https://captcha-backgrounds.s3.yandex.net/static/kinopoisk-background.jpg')}}.LogoLink{background-image:url('')}</style></head><body><div id="root"><div class="Theme Theme_color_yandex-default Theme_root_default"><div class="Container"><div c

0


In [55]:
data = pd.DataFrame(data)

In [56]:
data

,Kinopoisk_Link,Name,Name_ENG,Year,Country,Genre,Director,Length,Rating
0,https://www.kinopoisk.ru/film/435/,Зеленая миля,The Green Mile,1999,США,драма,Фрэнк Дарабонт,189,9.2
1,https://www.kinopoisk.ru/film/329/,Список Шиндлера,Schindler's List,1993,США,драма,Стивен Спилберг,195,9.1
2,https://www.kinopoisk.ru/film/326/,Побег из Шоушенка,The Shawshank Redemption,1994,США,драма,Фрэнк Дарабонт,142,9.0
3,https://www.kinopoisk.ru/film/448/,Форрест Гамп,Forrest Gump,1994,США,драма,Роберт Земекис,142,9.0
4,https://www.kinopoisk.ru/film/679486/,Тайна Коко,Coco,2017,США,мультфильм,Ли Анкрич,105,8.9
...,...,...,...,...,...,...,...,...,...
95,https://www.kinopoisk.ru/film/437410/,Темный рыцарь: Возрождение легенды,The Dark Knight Rises,2012,США,фантастика,Кристофер Нолан,165,8.4
96,https://www.kinopoisk.ru/film/278522/,Хоббит: Нежданное путешествие,The Hobbit: An Unexpected Journey,2012,Зеландия,фэнтези,Питер Джексон,169,8.4
97,https://www.kinopoisk.ru/film/81314/,Отступники,The Departed,2006,США,триллер,Мартин Скорсезе,151,8.4
98,https://www.kinopoisk.ru/film/399/,Храброе сердце,Braveheart,1995,США,история,Мэл Гибсон,178,8.4


пока в нашем случае разные агенты только ухудшили сбор данных.

#### Rotating IPs

In [57]:
from itertools import cycle
import traceback

In [72]:
browser = webdriver.Chrome(r'C:\Users\User\Desktop\chromedriver.exe')
url = 'http://free-proxy.cz/ru/proxylist/country/all/https/ping/level1'
# url = 'http://free-proxy.cz/ru/proxylist/country/RU/all/ping/all'
browser.get(url) 

all_prxs = len(browser.find_elements_by_tag_name('tr'))
proxies = []
port = []

for i in range(all_prxs):
    try:
        total_txt = browser.find_element_by_xpath(f'//*[@id="proxy_list"]/tbody/tr[{i+1}]/td[1]').text
        child_txt = browser.find_element_by_xpath(f'//*[@id="proxy_list"]/tbody/tr[{i+1}]/td[1]/script').text
        proxy = total_txt.replace(child_txt, '')
        port.append(browser.find_element_by_xpath(f'//*[@id="proxy_list"]/tbody/tr[{i+1}]/td[2]/span').text)
        proxies.append(proxy)
        
    except NoSuchElementException:
        pass

if len(proxies) == len(port):
    proxies = dict(zip(proxies, port))
else:
     print('len(proxies) != len(port). Собраны не все данные по прокси или порту.')
        
proxx = set()
for i,j in proxies.items():
    proxx.add(':'.join([i, j]))
#     i + ':' + j


print(len(proxies), len(proxx))

{'100.2.236.107': '80', '185.245.182.5': '80', '5.78.50.231': '8888', '104.223.135.178': '10000', '46.167.209.97': '3128', '2.106.185.11': '80', '167.172.172.234': '35855', '157.230.241.133': '36161', '167.71.199.211': '45865', '161.35.214.127': '36557', '46.101.126.180': '42345', '191.103.219.225': '48612', '5.189.184.6': '80', '122.176.48.148': '80', '167.172.173.210': '44627', '149.28.53.45': '38080', '114.130.39.169': '8080', '64.251.22.220': '80', '176.31.68.255': '20236', '116.98.176.52': '10003', '1.234.23.159': '80', '51.38.230.146': '443', '95.156.230.45': '80', '123.30.154.38': '2008', '115.144.102.39': '10080', '128.199.20.88': '8080', '68.183.230.116': '37299', '139.5.64.108': '80', '95.0.84.26': '80', '116.254.125.237': '9091', '191.36.151.129': '8941', '168.235.69.34': '80', '172.107.97.51': '443', '51.222.254.3': '80', '62.171.189.81': '80'} {'167.172.172.234:35855', '157.230.241.133:36161', '168.235.69.34:80', '1.234.23.159:80', '5.189.184.6:80', '161.35.214.127:36557',

In [73]:
print(len(proxies), len(proxx))

35 35


In [ ]:
proxy_pool = cycle(proxx)

# url = 'https://www.kinopoisk.ru/lists/movies/top250/'
url = 'https://www.apple.com/'
for i in range(1, len(proxx)):
    # get a proxy from the pool
    proxy = next(proxy_pool)
    print("Request #%d"%i)
    try:
        response = requests.get(url, proxies=proxy)
        print(response.json())
    except:
        # most free proxies get connection errors. You will have retry the entire request using another proxy to work.  
        print("Skipping. Connnection error")

In [ ]:
browser = webdriver.Chrome(options=options, executable_path=r'C:\Users\User\Desktop\chromedriver.exe')
url = 'https://www.kinopoisk.ru/lists/movies/top250/'
browser.get(url)
time.sleep(4)

while True:
    
    url = browser.current_url
    if url[:-1] == '4':
        user_agent = ua.random
        options.add_argument(f'user-agent={user_agent}')
        print(user_agent)
    
    time.sleep(3)
   
    url = check_captcha(browser, url, BeautifulSoup(browser.page_source, features='lxml'))
    print(url)
    
    r = requests.get(url, proxies=proxy)
    check_captcha(browser, url, BeautifulSoup(browser.page_source, features='lxml'))
    time.sleep(2)  # for captcha problem stabilization
    
    soup = BeautifulSoup(r.content, features='lxml')
    print(soup)
    time.sleep(2)  
    films = soup.find_all('div', class_='styles_upper__j8BIs')
    print(len(films))  # monitoring url response

    for film in films:
        link.append('https://www.kinopoisk.ru' + film.find('a', class_='base-movie-main-info_link__YwtP1').get('href'))
        name.append(film.find('span', class_='styles_mainTitle__IFQyZ styles_activeMovieTittle__kJdJj').text)

        rating.append(float(
                          film.find('span', 
                          class_='styles_kinopoiskValuePositive__vOb2E styles_kinopoiskValue__9qXjg styles_top250Type__mPloU').text)
                         )

        string = unicodedata.normalize("NFKD", film.find_all('span', 
                                                             class_='desktop-list-main-info_truncatedText__IMQRP')[0].text)
        director.append(re.compile("Режиссёр: (.*)$").search(string).group(1))
        
        time.sleep(1)
        
        string = unicodedata.normalize("NFKD", film.find_all('span', 
                                                             class_='desktop-list-main-info_truncatedText__IMQRP')[0].text)
        country.append(re.compile('(\w+) •').search(string).group(1))

        string = unicodedata.normalize("NFKD", film.find_all('span', 
                                                             class_='desktop-list-main-info_truncatedText__IMQRP')[0].text)
        genre.append(re.compile('[^  ]*').search(re.compile('• (.*)$').search(string).group(1)).group(0))

        time.sleep(1)
        
        data_block = film.find('div', class_='desktop-list-main-info_secondaryTitleSlot__mc0mI').find_all('span')
        
        if len(data_block) == 2:

            name_eng.append(data_block[0].text)
            string = unicodedata.normalize("NFKD", data_block[1].text)
            year.append(int(string.split(', ')[1]))
            minutes_time.append(int(re.compile('[0-9]+').search(string.split(', ')[2]).group(0)))

        else:
            
            name_eng.append('---')
            string = unicodedata.normalize("NFKD", data_block[0].text)
            year.append(int(string.split(', ')[0]))
            minutes_time.append(int(re.compile('[0-9]+').search(string.split(', ')[1]).group(0)))
    
    data = {'Kinopoisk_Link': link, 'Name': name, 'Name_ENG': name_eng,
            'Year': year, 'Country': country, 'Genre': genre,
            'Director': director, 'Length': minutes_time, 'Rating': rating}

    time.sleep(10)
    try:  # here we navigate the pages
        next_page_button = browser.find_element_by_xpath('//a[@title="Вперед"]')
        next_page_button.click()
        time.sleep(3)

    except NoSuchElementException:
        browser.quit()
        break